In [1]:
import glob
import pandas as pd
from zipfile import ZipFile

import sqlite3
from sqlalchemy import create_engine, func, inspect, distinct
import os

from sqlalchemy.ext.declarative import declarative_base



In [3]:
files = glob.glob('/Users/claytonhribar/Documents/Citi-bike-project/Data/2020_data/*.zip')

In [4]:
table = []



for i in files:
    

    zf = ZipFile(i)
    
    name = zf.namelist()[0]
    
    print(name)

    df = pd.read_csv(zf.open(name))
    
    #df['starttime'] = pd.to_datetime(df.starttime)
    
    #df = df.set_index(['starttime'])
    
    table.append(df)

202004-citibike-tripdata.csv
202003-citibike-tripdata.csv
202006-citibike-tripdata.csv
202001-citibike-tripdata.csv
202008-citibike-tripdata.csv
202007-citibike-tripdata.csv
202002-citibike-tripdata.csv
202005-citibike-tripdata.csv


In [5]:
bike_df = pd.concat(table)

In [6]:
bike_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,534,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,3656,E 2 St & Avenue A,40.723077,-73.985836,545,E 23 St & 1 Ave,40.736502,-73.978095,15162,Subscriber,1990,2
1,529,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,3163,Central Park West & W 68 St,40.773407,-73.977825,3301,Columbus Ave & W 95 St,40.791956,-73.968087,42861,Subscriber,1974,1
2,1488,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,3164,Columbus Ave & W 72 St,40.777057,-73.978985,3605,31 Ave & Crescent St,40.765835,-73.926547,35989,Subscriber,1982,1
3,341,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,3827,Halsey St & Broadway,40.685650,-73.915640,3869,Van Buren St & Broadway,40.692000,-73.926170,37548,Subscriber,1996,1
4,2206,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,3890,Grove St & Broadway,40.690470,-73.923511,3058,Lewis Ave & Kosciuszko St,40.692371,-73.937054,39701,Subscriber,1977,1


In [7]:
len(bike_df.index)

11944130

In [8]:
len(bike_df['start station name'].unique())

1076

In [252]:
temp = bike_df.drop(columns = ['tripduration', 'starttime', 'stoptime','end station id', 'end station name', 'end station latitude', 'end station longitude','bikeid', 'usertype','birth year', 'gender'])
temp2 = temp.sort_values(['start station id'])

In [253]:
station_info = pd.DataFrame(temp2.drop_duplicates(subset = ['start station name'])).reindex()

In [254]:
station_info = station_info.rename(columns = {'start station id': 'station_id', 'start station name':'station_location','start station latitude':'latitude', 'start station longitude':'longitude'})

In [256]:
station_info = station_info.drop_duplicates(subset = ['station_id'])

In [257]:
station_info

,station_id,station_location,latitude,longitude
579312,72,W 52 St & 11 Ave,40.767272,-73.993929
853949,79,Franklin St & W Broadway,40.719116,-74.006667
896319,82,St James Pl & Pearl St,40.711174,-74.000165
1242341,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323
933531,116,W 17 St & 8 Ave,40.741776,-74.001497
...,...,...,...,...
2266532,4110,Avenue D & E 10 St,40.725066,-73.975158
2037427,4116,Gerard Ave & E 164 St,40.830262,-73.923391
1698600,4122,Central Park W & W 97 St,40.792495,-73.964171
1646994,4123,Central Park W & W 103 St,40.796037,-73.961589


In [297]:
rides_df = pd.DataFrame(bike_df.drop(columns=['start station name','stoptime', 'start station latitude', 'start station longitude', 'end station name', 'end station latitude', 'end station longitude','bikeid']))

In [298]:
rides_df['starttime'] = pd.to_datetime(rides_df['starttime'])
#rides_df['stoptime']=pd.to_datetime(rides_df['stoptime'])


In [299]:
rides_df['ride_no'] = range(1, len(rides_df) + 1)

In [300]:
rides_df.dtypes

tripduration                 int64
starttime           datetime64[ns]
start station id             int64
end station id               int64
usertype                    object
birth year                   int64
gender                       int64
ride_no                      int64
dtype: object

In [315]:
rides_df = rides_df[['ride_no','tripduration','starttime','start_station_id','end_station_id','usertype','birth_year','gender']]

In [316]:
rides_df = rides_df.rename(columns={'start station id': 'start_station_id', 'end station id': 'end_station_id', 'birth year': 'birth_year'})

In [317]:
rides_df = rides_df[['ride_no','tripduration','starttime','start_station_id','end_station_id','usertype','birth_year','gender']]

In [318]:
rides_df

,ride_no,tripduration,starttime,start_station_id,end_station_id,usertype,birth_year,gender
0,1,534,2020-04-01 00:00:15.280,3656,545,Subscriber,1990,2
1,2,529,2020-04-01 00:02:28.943,3163,3301,Subscriber,1974,1
2,3,1488,2020-04-01 00:02:56.090,3164,3605,Subscriber,1982,1
3,4,341,2020-04-01 00:03:25.141,3827,3869,Subscriber,1996,1
4,5,2206,2020-04-01 00:05:35.279,3890,3058,Subscriber,1977,1
...,...,...,...,...,...,...,...,...
1487885,11944126,981,2020-05-31 23:59:19.031,313,217,Subscriber,1972,2
1487886,11944127,367,2020-05-31 23:59:36.227,545,501,Subscriber,1994,2
1487887,11944128,1798,2020-05-31 23:59:37.725,450,485,Subscriber,1998,1
1487888,11944129,718,2020-05-31 23:59:47.709,3691,3691,Customer,1969,0


In [353]:
create_rides_table = """ CREATE TABLE rides_df (
                        [ride_no] LONG NOT NULL PRIMARY KEY,
                        [tripduration] integer,
                        [starttime] DATETIME,
                        [start_station_id] integer,
                        [end_station_id] integer,
                        [usertype] text,
                        [birth_year] integer,
                        [gender] integer,
                        FOREIGN KEY ('start_station_id') REFERENCES station_info ('station_id')
                        FOREIGN KEY ('end_station_id') REFERENCES station_info ('station_id') );
                        """

In [354]:
create_stations_table = """ CREATE TABLE station_info (
                            [station_id] INTEGER NOT NULL PRIMARY KEY,
                            [station_location] TEXT,
                            [latitude] REAL,
                            [longitude] REAL
                            );"""

In [392]:
conn = sqlite3.connect('sql_1.sqlite3')

c = conn.cursor()

In [373]:
def get_df_name(df):
    name = [x for x in globals() if globals()[x] is df]
    name = name[0]
    
    return name


In [393]:
#@note; make sure names of dfs are retunred correctly. might have to mess around w/ index of get_df_name function
#
#do no call either table after you have names displayed correctly

get_df_name(rides_df)

'rides_df'

In [394]:
#@note; make sure names of dfs are retunred correctly. might have to mess around w/ index of get_df_name function


get_df_name(station_info)

'station_info'

In [395]:
###

def create_sql_table(x):
    
    c.execute(x)
    
    conn.commit()
    


In [396]:
create_sql_table(create_stations_table)

In [397]:
create_sql_table(create_rides_table)

In [398]:
def add_data_to_table(x):
    
    y = get_df_name(x)
    
    x.to_sql(y, conn, if_exists='append', index=False)    #, if_exists= 'replace', index = False)

In [399]:
add_data_to_table(station_info)

In [400]:
add_data_to_table(rides_df)

In [401]:

def sql_fetch():

    cursorObj = conn.cursor()

    cursorObj.execute('SELECT name from sqlite_master where type= "table"')

    print(cursorObj.fetchall())

sql_fetch()

[('station_info',), ('rides_df',)]


In [ ]:
#not used

"""

def create_sql_2(data, holder):
    
    
    
    q = get_df_name(data)
    
    c.execute(holder)
    
    conn.commit()
    
    data.to_sql(q , conn, if_exists='replace', index = False)
"""

In [ ]:
# not used

""" 
def column_names(df):
    all_cols = []
    col_list = df.columns.to_list()
    for i in col_list:
        clean_item = str(i).replace(" ","_")
        all_cols.append(clean_item)
            
    final_list = str(all_cols).replace("[","(").replace("'","").replace("]",")").replace('"','').replace("-","").replace(".","pnt")
    
    
    return final_list



def new_table(holder, table_name, data):
    
    y = holder
    
    q = table_name
    
    f = str(f"'CREATE TABLE {q} {y}'")
    
    #f = str(f"'CREATE TABLE {q} id integer PRIMARY KEY, {y[1:]}'")
    
    g = holder
    
    c.execute(g)
    
    conn.commit()
    
    data.to_sql(q , conn, if_exists='replace', index = False)
    
def add_to_sql(x):
    
    y = get_df_name(x)
    
    x.to_sql(y, conn, if_exists= 'replace', index = False)
    
"""